<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2022notebooks/2022_0506pytorch_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch の基礎

- Date: 2022-0506
- Author: 浅川伸一 asakawa@ieee.org

## PyTorch の特徴
<!-- ## Features of PyTorch-->

- 実行による定義 `define by run` (Chainer から継承) 記号的プログラミングではなく，命令的プログラミング
- 自動微分 Atuorgrad
- 最近のエコシステムで多くサポートされている、例えば
    - GPU フレンドリ
    - numpy フレンドリ

- [Stackoverflow の言語トレンド](https://insights.stackoverflow.com/trends?tags=python%2Cjavascript%2Cjava%2Cc%23%2Cphp%2Cc%2B%2B)
- [TensorFlow と PyTorch の人気比較](http://horace.io/pytorch-vs-tensorflow/)
- [Google トレンド](https://trends.google.com/trends/explore?q=pytorch,keras,tensorflow)

## 前提知識 (常識的な意味で)

- [Google colab](https://colab.research.google.com/notebooks/intro.ipynb), [jupyter notebook](https://jupyter.org/) のクラウドコンピューティング実装.  
    - `Jupyter notebook` は [ipython](https://ipython.org/) の Web 版実装.  
    - Ipython  [Python](https://www.python.org/) は対話型実行環境.
    - Python は reddit や stackoverflow などの機械学習コミュニティで最も人気のあるプログラミング言語の 1 つです。

- Python を知らない？ それはもう問題ではありません。
- 機械学習 を知らない？ もう選択の余地がありません。
- ディープラーニング を知らない？ もう常識です

<!-- - Not familiar with Python?  It doesn't matter now.
- Not familiar with machine learning? It doesn't matter now.
- Not familiar with deep learning? It's the main subject of today's talk!  You can feel ease with it when your going to bed tonight.
-->


## jupyter notebook の使用法
<!-- ## How to use jupyter notebook.-->

- [Chrome](https://www.google.com/chrome/) ブラウザ上で`colab`を実行する必要があります。 
OS は問いません。
スマートフォンでも `Colab` ファイルを実行することができます。
- `Colab` は Chrom の タブ内にある複数のセルから構成されています。
- セルを実行するにはシフトキーを押しながらエンターキーを押す。
または，セル左上の 黒丸の中に白い三角` のアイコンを押下します。


## 参考

- ツィッター: https://twitter.com/PyTorch
- Forum: https://discuss.pytorch.org/
- チュートリアル: https://pytorch.org/tutorials/
- コード例: https://github.com/pytorch/examples
- API Reference: https://pytorch.org/docs/stable/index.html
- Torchvision: https://pytorch.org/docs/stable/torchvision/index.html
- PyTorch Text: https://github.com/pytorch/text
- PyTorch Audio: https://github.com/pytorch/audio
- AllenNLP: https://allennlp.org/
- 物体検出/領域切り出し: https://github.com/facebookresearch/maskrcnn-benchmark
- フェイスブック人工知能研究所 (FAIR) Sequence-to-Sequence Toolkit written in PyTorch: https://github.com/pytorch/fairseq
- FastAI http://www.fast.ai/
- スタンフォード大学のディープラーニング講義ノート https://cs230-stanford.github.io
- OpenAI: https://openai.com/research/


## 初期設定 Initialization

In [ ]:
import numpy as np 
# ほぼすべてのニューラルネットワーク，機械学習系ライブラリは numpy および scipy に依存しています

import matplotlib.pyplot as plt
%matplotlib inline
# 描画ライブラリ

import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    !pip install japanize_matplotlib
import japanize_matplotlib

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
np.eye(3)  # numpy での単位行列

In [ ]:
torch.eye(3)  # PyTorch での単位行列

In [ ]:
X = np.random.random([4, 3])  # numpy による 4 行 3 列の乱数行列
X

In [ ]:
X_ = torch.rand([4, 3])  # PyTorch による 4 行 3 列の乱数行列
X_

In [ ]:
#X.shape
X.ndim

In [ ]:
from sklearn.datasets import make_regression

n_features = 1
n_samples = 100

X, y = make_regression(
    n_samples=n_samples,
    n_features=n_features,
    noise=20,
    random_state=42,
)

fix, ax = plt.subplots()
ax.set_title('回帰分析結果')
ax.plot(X, y, ".")


In [9]:
X = torch.from_numpy(X).float()
y = torch.from_numpy(y.reshape((n_samples, n_features))).float()

In [10]:
class LinReg(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.beta = nn.Linear(input_dim, 1)
        
    def forward(self, X):
        return self.beta(X)

In [11]:
model = LinReg(n_features).to(DEVICE)
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
X, y = X.to(DEVICE), y.to(DEVICE)

In [ ]:
# Train step
model.train()  # 訓練モード，すなわちパラメータの更新を行う
optimizer.zero_grad()

y_ = model(X)
loss = loss_fn(y_, y)

loss.backward()   # 一度だけ学習を行うため損失値を計算
optimizer.step()  # 損失値に基づいてパラメータの更新，すなわち学習

# Eval
model.eval()  # 評価モード，すなわちパラメータの更新を行わない
with torch.no_grad():
    y_ = model(X)    

# 結果の視覚化
fig, ax = plt.subplots()
ax.plot(X.cpu().numpy(), y_.cpu().numpy(), ".", label="予測値")
ax.plot(X.cpu().numpy(), y.cpu().numpy(), ".", label="データ")
ax.set_title(f"平均自乗誤差 MSE: {loss.item():0.1f}")
ax.legend();

In [ ]:
# 学習させてみる
model.train()
losses = []
for i in range(20):
    _y = model(X)
    loss = loss_fn(_y, y)
    losses.append(loss.item())
    print(loss.item())
    loss.backward()
    optimizer.step()

# 結果の視覚化
fig, ax = plt.subplots()
ax.plot(X.cpu().numpy(), _y.detach().numpy(), ".", label="予測値")
ax.plot(X.cpu().numpy(), y.detach().numpy(), ".", label="データ")
ax.set_title(f"MSE: {loss.item():0.1f}")
ax.legend();    

In [ ]:
print(losses)